### Fetch the dataset

In [134]:
import pandas as pd

df = pd.read_csv("clean_coffee_shop_sales.csv")
df.head()

,Unnamed: 0,transaction_id,transaction_date,transaction_time,transaction_qty,store_id,store_location,product_id,unit_price,product_category,product_type,product_detail,transaction_month
0,0,1,2023-01-01,07:06:11,2,5,Lower Manhattan,32,3.0,Coffee,Gourmet brewed coffee,Ethiopia Rg,1
1,1,2,2023-01-01,07:08:56,2,5,Lower Manhattan,57,3.1,Tea,Brewed Chai tea,Spicy Eye Opener Chai Lg,1
2,2,3,2023-01-01,07:14:04,2,5,Lower Manhattan,59,4.5,Drinking Chocolate,Hot chocolate,Dark chocolate Lg,1
3,3,4,2023-01-01,07:20:24,1,5,Lower Manhattan,22,2.0,Coffee,Drip coffee,Our Old Time Diner Blend Sm,1
4,4,5,2023-01-01,07:22:41,2,5,Lower Manhattan,57,3.1,Tea,Brewed Chai tea,Spicy Eye Opener Chai Lg,1


In [135]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149116 entries, 0 to 149115
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Unnamed: 0         149116 non-null  int64  
 1   transaction_id     149116 non-null  int64  
 2   transaction_date   149116 non-null  object 
 3   transaction_time   149116 non-null  object 
 4   transaction_qty    149116 non-null  int64  
 5   store_id           149116 non-null  int64  
 6   store_location     149116 non-null  object 
 7   product_id         149116 non-null  int64  
 8   unit_price         149116 non-null  float64
 9   product_category   149116 non-null  object 
 10  product_type       149116 non-null  object 
 11  product_detail     149116 non-null  object 
 12  transaction_month  149116 non-null  int64  
dtypes: float64(1), int64(6), object(6)
memory usage: 14.8+ MB


### Create a new dataset

In [136]:
month = df['transaction_month'].unique()
month

array([1, 2, 3, 4, 5, 6], dtype=int64)

In [137]:
data = {
    'transaction_month': month
}

df_monthly = pd.DataFrame(data)

In [138]:
df_monthly

,transaction_month
0,1
1,2
2,3
3,4
4,5
5,6


### Calculate the total sales for each respective month.

In [139]:
# create a function that solves for the total sales in a given month and year
def total_monthly_sales (month):
    try:
        # Filter the DataFrame for the specified month and year
        subset_df = df[df['transaction_month'] == month]

        # Calculate the total sales and round it to 2 decimal places
        total_sales = (subset_df['unit_price']*subset_df['transaction_qty']).sum()
        total_sales = round(total_sales, 2)

        return total_sales
    
    except Exception as e:
        print(f"An error occured: {str(e)}")
        return None

In [140]:
# try using the function
total_monthly_sales (5)

156727.76

In [141]:
# Initialize an empty list to store total sales for each month
total_sales = []

# Iterate over each row in the DataFrame df_monthly using iterrows()
for index, row in df_monthly.iterrows():
    # For each row, call the total_monthly_sales function with the current month
    # This function calculates the total sales for the specified month
    sales_for_month = total_monthly_sales(row['transaction_month'])
    
    # Append the result of the total sales for the current month to the total_sales list
    total_sales.append(sales_for_month)

# After the loop, add the total_sales list as a new column in the df_monthly DataFrame
# This new column will contain the total sales for each corresponding month
df_monthly['total_sales'] = total_sales

In [142]:
df_monthly

,transaction_month,total_sales
0,1,81677.74
1,2,76145.19
2,3,98834.68
3,4,118941.08
4,5,156727.76
5,6,166485.88


### Calculate the difference in sales between the selected month and the previous month.

Version 1:
Create a function and manually get the monthly sales difference

In [143]:
def monthly_sales_difference(month):
    try:
        # Check if the month is the first month (January)
        if month == 1:
            return None  # No previous month to compare, return None
        else:
            # Get total sales for the current month from the DataFrame
            total_sales_current = df_monthly[df_monthly['transaction_month'] == month]['total_sales'].values[0]
            
            # Get total sales for the previous month from the DataFrame
            total_sales_previous = df_monthly[df_monthly['transaction_month'] == month - 1]['total_sales'].values[0]

            # Calculate the difference in sales between the current and previous month
            sales_difference = total_sales_current - total_sales_previous
            
            # Round the sales difference to two decimal places
            sales_difference = round(sales_difference, 2)
            
            # Return the computed sales difference
            return sales_difference
    except Exception as e:
        # Print an error message if an exception occurs
        print(f"An error occurred: {str(e)}")


In [144]:
# Initialize an empty list to store sales_difference for each month
sales_difference = []

# Iterate over each row in the DataFrame df_monthly using iterrows()
for index, row in df_monthly.iterrows():
    sales_difference_month = monthly_sales_difference(row['transaction_month'])
    
    sales_difference.append(sales_difference_month)

sales_difference

[None, -5532.55, 22689.49, 20106.4, 37786.68, 9758.12]

Version 2: Use the pandas function

In [145]:
df_monthly['sales_difference'] = df_monthly['total_sales'].diff()
df_monthly

,transaction_month,total_sales,sales_difference
0,1,81677.74,NaN
1,2,76145.19,-5532.55
2,3,98834.68,22689.49
3,4,118941.08,20106.40
4,5,156727.76,37786.68
5,6,166485.88,9758.12


### Determine the month-on-month % increase or decrease in sales.

Version 1:

In [146]:
def percent_change_monthly(month):
    try:
        # Check if the month is the first month (January)
        if month == 1:
            return None  # No previous month to compare, return None
        else:
            # Get total sales for the current month from the DataFrame
            total_sales_current = df_monthly[df_monthly['transaction_month'] == month]['total_sales'].values[0]
            
            # Get total sales for the previous month from the DataFrame
            total_sales_previous = df_monthly[df_monthly['transaction_month'] == month - 1]['total_sales'].values[0]

            percent_change = ((total_sales_current-total_sales_previous)/total_sales_previous)*100
            
            # Round the sales percent difference to two decimal places
            percent_change = round(percent_change, 2)
            
            # Return the computed sales percent difference
            return percent_change
    except Exception as e:
        # Print an error message if an exception occurs
        print(f"An error occurred: {str(e)}")


In [147]:
# Initialize an empty list to store sales_difference for each month
sales_percent_change = []

# Iterate over each row in the DataFrame df_monthly using iterrows()
for index, row in df_monthly.iterrows():
    sales_percent_month = percent_change_monthly(row['transaction_month'])
    
    sales_percent_change.append(sales_percent_month)

sales_percent_change

[None, -6.77, 29.8, 20.34, 31.77, 6.23]

Version 2:

In [148]:
df_monthly['sales_percent_change'] = df_monthly['total_sales'].pct_change() * 100  
# Multiply by 100 to convert to percentage
df_monthly

,transaction_month,total_sales,sales_difference,sales_percent_change
0,1,81677.74,NaN,NaN
1,2,76145.19,-5532.55,-6.773633
2,3,98834.68,22689.49,29.797667
3,4,118941.08,20106.40,20.343466
4,5,156727.76,37786.68,31.769242
5,6,166485.88,9758.12,6.226159


### Export Monthly File

In [149]:
df.to_csv("monthly_sales_summary.csv")